In [3]:
import numpy as np
import scipy
import pandas as pd


In [4]:
#GLOBAL VARIABLES
DS_Distance_matrix = np.zeros((4096,100))
DS_matrix = np.zeros_like(DS_Distance_matrix)
dh = pd.read_csv('Demand_History.csv')
df  = dh.iloc[:,[0,1,2,-1]] #I am currently using 2018's data.
su = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
ps_array = np.array(su.loc[:,'total_parking_slots'])
scs_existing = np.array(su.loc[:,'existing_num_SCS'])
fcs_existing = np.array(su.loc[:,'existing_num_FCS'])
fcs_array_0 = fcs_existing.copy()
scs_array_0 = scs_existing.copy()
#HELPER FUNCTIONS
def flatten(fcs_array, scs_array, DS_matrix):
  one_long_input = np.concatenate((fcs_array,scs_array,DS_matrix.flatten()))
    
  return one_long_input
def unflatten(one_long_input):
  fcs_array = one_long_input[0:100]
  one_long_input = one_long_input[100:]
  scs_array = one_long_input[0:100]
  one_long_input = one_long_input[100:]
  DS_matrix = np.reshape(one_long_input, (4096,100))
  return fcs_array,scs_array, DS_matrix
#a,b,c = unflatten(flatten(fcs_array_0,scs_array_0,DS_matrix))
init_guesses = flatten(fcs_array_0,scs_array_0, DS_matrix)



def create_DS_Distance_matrix():
  global DS_Distance_matrix
  for row in df.index:
    for col in range(len(df.columns)):
      DS_Distance_matrix[row][col] = df.loc[row].iloc[col]
create_DS_Distance_matrix()  
  
def customer_dissatisfaction(DS_matrix):
  cost = 0
  for row in range(len(DS_matrix)):
    for col in range(len(DS_matrix[0])):
      cost += DS_matrix[row][col]*DS_Distance_matrix[row][col]
  return cost
def infrastructure_cost(fcs_array,scs_array):
  cost = 0
  for i in fcs_array:
    cost += i*1.5
  for i in scs_array:
    cost += i
  return cost


In [6]:
#OBJECTIVE
def objective(one_long_input):
  fcs_array , scs_array,DS_matrix = unflatten(one_long_input)
  cost1 = customer_dissatisfaction(DS_matrix)
  cost2 = infrastructure_cost(fcs_array, scs_array)
  return cost1+ 600*cost2

cons=[]
def all_constraints(one_long_input):
  fcs_array , scs_array,DS_matrix = unflatten(one_long_input)
  if(sum(n < 0  for rows in  DS_matrix for n in rows) !=0):
    return 1
  if(sum(n <0 for n in fcs_array )!=0):
    return 1
  if(sum(n <0 for n in scs_array )!=0):
    return 1
  for i in range(len(fcs_array)):
    if( (fcs_array[i] + scs_array[i] != ps_array[i]) or (fcs_array[i]  < fcs_existing[i]) or (scs_array[i]  < scs_existing[i]) ):
      return 1
  #extra constraints
  for row in DS_matrix:
    for val in row:
      if(val>1):
        return 1
  return 0



cons.append({'type':'eq','fun':all_constraints})



'''
COlumn of the DS_matrix must add up to corresponding total supply of the parking location (scs*200 + fcs*400)
'''

'\nCOlumn of the DS_matrix must add up to corresponding total supply of the parking location (scs*200 + fcs*400)\n'

In [ ]:
from scipy.optimize import minimize
result = minimize(objective, init_guesses,constraints=cons)

In [ ]:
import numpy as np
nop = np.array([[0,1,2],[1,2,3]])
sum(n<0 for rows in nop for n in rows)